In [1]:
import gzip

import numpy as np
import tensorflow as tf

from binary_sayc import load_deals, DealData
from simulator import Simulator

### Restore SAYC model

In [2]:
sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph('../model/sayc-500000.meta')
saver.restore(sess, '../model/sayc-500000')

graph = tf.get_default_graph()

seq_in = graph.get_tensor_by_name('seq_in:0')
keep_prob = graph.get_tensor_by_name('keep_prob:0')

out_bid_logit = graph.get_tensor_by_name('out_bid_logit:0')

INFO:tensorflow:Restoring parameters from ../model/sayc-500000


In [3]:
def model(sess):
    def pred_fun(X):
        bids = sess.run(out_bid_logit, feed_dict={seq_in: X, keep_prob: 0.95})
        return sess.run(tf.nn.softmax(bids[-1]))
    return pred_fun

In [4]:
reader = load_deals(gzip.open('../deals_bidding_valid_0001.gz'))

In [5]:
for _ in range(700): next(reader)

In [6]:
deal_str, auction_str, contracts = next(reader)

In [7]:
deal_str

'SA82 HK532 DAKQ9 C54  SK HAQT87 DJ8654 CQ6  ST5 HJ6 DT3 CAKJ9873  SQJ97643 H94 D72 CT2'

In [8]:
auction_str

'N Both 1N P 3C P P P'

In [9]:
deal_data = DealData.from_deal_auction_string(deal_str, auction_str)
deal_data.reset_auction()

In [10]:
init_auction = deal_data.auction
init_auction

[]

In [11]:
sim = Simulator(deal_data, contracts, model(sess))

In [12]:
sim.simulate_auction(init_auction)

['1N', 'PASS', '3C', 'PASS', 'PASS', 'PASS']

In [13]:
deal_str

'SA82 HK532 DAKQ9 C54  SK HAQT87 DJ8654 CQ6  ST5 HJ6 DT3 CAKJ9873  SQJ97643 H94 D72 CT2'

In [14]:
sim.best_auction(init_auction, 500)

(-690.0, ['1N', 'PASS', '3N', 'PASS', 'PASS', 'PASS'])

In [56]:
contracts['2SE']

(170, 170)

In [50]:
set(sim.simulate_bid([], 10000))

{'1H', 'PASS'}